In [1]:
import ee
import folium
from IPython.display import Image,display
from geopy.geocoders import Nominatim
import io
from PIL import Image

from selenium_driverless import webdriver
from selenium_driverless.types.by import By
import time

import cv2
import numpy as np

In [2]:
ee.Authenticate()
ee.Initialize()

Enter verification code: 4/1AeaYSHAfZUZX_wQd2CvA6Jrr5bnlMb4jVCNVdmxgc8KNqph1rLd8f3YZ_rI

Successfully saved authorization token.


In [3]:
def add_ee_layer(self, ee_object, vis_params, name):
    try:
        if isinstance(ee_object, ee.image.Image):    
            map_id_dict = ee.Image(ee_object).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        elif isinstance(ee_object, ee.imagecollection.ImageCollection):    
            ee_object_new = ee_object.mosaic()
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        elif isinstance(ee_object, ee.geometry.Geometry):    
            folium.GeoJson(
            data = ee_object.getInfo(),
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
        elif isinstance(ee_object, ee.featurecollection.FeatureCollection):  
            ee_object_new = ee.Image().paint(ee_object, 0, 2)
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
    
    except:
        print("Could not display {}".format(name))
    

folium.Map.add_ee_layer = add_ee_layer
def add_ee_layer(self, ee_object, vis_params, name):
    
    try:
        if isinstance(ee_object, ee.image.Image):    
            map_id_dict = ee.Image(ee_object).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        elif isinstance(ee_object, ee.imagecollection.ImageCollection):    
            ee_object_new = ee_object.mosaic()
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        elif isinstance(ee_object, ee.geometry.Geometry):    
            folium.GeoJson(
            data = ee_object.getInfo(),
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
        elif isinstance(ee_object, ee.featurecollection.FeatureCollection):  
            ee_object_new = ee.Image().paint(ee_object, 0, 2)
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
    
    except:
        print("Could not display {}".format(name))
    
folium.Map.add_ee_layer = add_ee_layer

In [4]:
forest_cover = ee.Image('UMD/hansen/global_forest_change_2020_v1_8').select('treecover2000')
# forest_cover = ee.Image('UMD/hansen/global_forest_change_2020_v1_8')

In [11]:
geolocator = Nominatim(user_agent='my')
city = 'bhubaneswar'
location = geolocator.geocode(city)

In [25]:
mapp = folium.Map(location=[location.latitude, location.longitude], zoom_start=11, no_touch=True, zoom_control=False)
mapp.add_ee_layer(forest_cover, {'palette': ['000000', '00FF00']}, 'Forest area')
mapp.save('map.html')

In [26]:
options = webdriver.ChromeOptions()
options.headless = True
async with webdriver.Chrome(options=options) as driver:
    await driver.get('file:///Users/xy0ke/Desktop/treeproj/map.html')
    await driver.sleep(3)
    await driver.save_screenshot('forest.png')
    await driver.quit()

print('saved!')

saved!


In [27]:
display(mapp)

In [15]:
img = cv2.imread("forest.png")

green = [20, 60, 0]

diff = 20
diff2 = 50

boundaries = [([green[2], green[1]-diff2, green[0]-diff],
           [green[2]+diff, green[1]+diff2, green[0]+diff])]

scalePercent = 1

width = int(img.shape[1] * scalePercent)
height = int(img.shape[0] * scalePercent)
newSize = (width, height)

img = cv2.resize(img, newSize, None, None, None, cv2.INTER_AREA)

for (lower, upper) in boundaries:

    lower = np.array(lower, dtype=np.uint8)
    upper = np.array(upper, dtype=np.uint8)

    mask = cv2.inRange(img, lower, upper)

    output = cv2.bitwise_and(img, img, mask=mask)

    ratio_green = cv2.countNonZero(mask)/(img.size/3)

    colorPercent = (ratio_green * 100) / scalePercent

    print('The percentage of trees in the area is:', np.round(colorPercent, 2))

The percentage of trees in the area is: 2.42


# Tree counter

In [20]:
geolocator = Nominatim(user_agent='my')
city = 'bhubaneswar'
location = geolocator.geocode(city)

x = location.latitude
y = location.longitude

# Define area of interest
# cord = "20.5648648,85.9923632"
# cord = cord.split(',')

# x = float(cord[0])
# y = float(cord[1])

th = 0.05

aoi = ee.Geometry.Polygon(
  [[[y+th, x-th],
    [y-th, x-th],
    [y-th, x+th],
    [y+th, x+th]]])

# Load imagery
image = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
          .filterDate('2020-01-01', '2020-12-31') \
          .median()

# Apply NDVI
ndvi = image.normalizedDifference(['B5', 'B4'])

# Apply threshold
threshold = ndvi.gt(0.3)

# Reduce to vectors
vectors = threshold.reduceToVectors(
  geometryType = 'polygon',
  scale = 30,
  geometry = aoi)

# Display on map
m = folium.Map(location=[x, y], zoom_start=13)
folium.GeoJson(vectors.getInfo()).add_to(m)
m

In [21]:
count = vectors.size()
print('Number of trees:', count.getInfo())

Number of trees: 1251
